In [1]:
import re
import sys
import glob
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
sns.set_style('darkgrid')

In [2]:
## parameters 
lowtrunc = 0
hightrunc = 7000
avgp = 15

train = pd.read_csv("train.csv")
train_frags = glob.glob("./train_fft/*")
len(train_frags)

all_df = pd.DataFrame() 
ertime = pd.Series(0, index=np.arange(len(train_frags)))

i = 0
for file in train_frags:
#     print('file',file)
    start = './train_fft/'
    end = '.csv'
    seg_id = file[file.find(start)+len(start):file.rfind(end)]
#     print('segment_id',seg_id)
    t2e = int(train.loc[train['segment_id'] == int(seg_id)]['time_to_eruption'].values)
#     print('time to eruption',t2e)


    ## let's combine the fft's of the sensors
    sequence = pd.read_csv(file)
    ## create composite of sensors frequenc domain
    composite = sequence.sum(axis=1) / len(sequence.columns)
    ## Truncate low frequency noise
    composite = composite[lowtrunc:-hightrunc]
    ## smooth signal with average
    composite = composite.rolling(avgp, win_type='triang').sum()
    composite = composite.fillna(0)
    ## min-max scale signal
    composite = (composite - composite.min()) / (composite.max() - composite.min())
#     print('time to eruption',t2e)
#     print('skew',composite.skew())
#     print('kurtosis',composite.kurt())
    ## Build DataFrame
    all_df = all_df.append(composite,ignore_index=True)
    ertime.iloc[i] = t2e
    i = i + 1

test_frags = glob.glob("./test_fft/*")
len(test_frags)

test_id = pd.Series(0, index=np.arange(len(test_frags)))

i = 0
for file in test_frags:
#     print('file',file)
    start = './test_fft/'
    end = '.csv'
    seg_id = file[file.find(start)+len(start):file.rfind(end)]

    ## let's combine the fft's of the sensors
    sequence = pd.read_csv(file)
    ## create composite of sensors frequenc domain
    composite = sequence.sum(axis=1) / len(sequence.columns)
    ## Truncate low frequency noise
    composite = composite[lowtrunc:-hightrunc]
    ## smooth signal with average
    composite = composite.rolling(avgp, win_type='triang').sum()
    composite = composite.fillna(0)
    ## min-max scale signal
    composite = (composite - composite.min()) / (composite.max() - composite.min())

    ## Build DataFrame
    all_df = all_df.append(composite,ignore_index=True)
    test_id.iloc[i] = seg_id
    i = i + 1
#     xf = np.linspace(0.0, 1.0/(2.0*1/100.0), len(composite))
#     plt.axis([0, 50, 0, 1])
#     plt.plot(xf, composite)
#     plt.grid()
#     plt.show()

In [3]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8951 entries, 0 to 8950
Columns: 23000 entries, 0 to 22999
dtypes: float64(23000)
memory usage: 1.5 GB


In [4]:
all_df.head()

,0,1,2,3,4,5,6,7,8,9,...,22990,22991,22992,22993,22994,22995,22996,22997,22998,22999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004130,0.004181,0.004194,0.004152,0.004054,0.003958,0.003888,0.003858,0.003847,0.003835
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005968,0.005897,0.005833,0.005827,0.005750,0.005750,0.005776,0.005803,0.005848,0.005925
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.005071,0.005034,0.004981,0.004913,0.004894,0.004869,0.004856,0.004795,0.004765,0.004874
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003003,0.003019,0.003031,0.003032,0.003043,0.003040,0.003029,0.003021,0.003021,0.003009
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.011714,0.011830,0.012011,0.012108,0.012332,0.012374,0.012593,0.012967,0.013336,0.013689


In [ ]:
npca = 55
from sklearn.decomposition import PCA
pca = PCA(n_components=npca,whiten=True)
pca_fit = pca.fit(all_df)

In [ ]:
all_pca = pca_fit.transform(all_df)

In [ ]:
out = pd.DataFrame(data=all_pca)
out.to_csv('pca_55.csv', index=False)